In [41]:
include("src/CombinedSubexpressions.jl")

CombinedSubexpressions

In [42]:
@CombinedSubexpressions.cse inv(H) * G + W * (inv(H) + W)

begin 
    ##invH#319 = inv(H)
    ##*##invH#319G#320 = ##invH#319 * G
    ##+##invH#319W#321 = ##invH#319 + W
    ##*W##+##invH#319W#321#322 = W * ##+##invH#319W#321
    ##+##*##invH#319G#320##*W##+##invH#319W#321#322#323 = ##*##invH#319G#320 + ##*W##+##invH#319W#321#322
    ##+##*##invH#319G#320##*W##+##invH#319W#321#322#323
end


LoadError: UndefVarError: H not defined

In [43]:
function g(x)
    println("g")
    1
end

function f(x)
    println("f")
    2
end

function h(x, y)
    println("h")
    3
end

x = 1

expr = quote
    function foo(x)
        [h(h(f(g(x)), g(x)), i) == i for i in 1:5]
    end
end

quote  # In[43], line 19:
    function foo(x) # In[43], line 20:
        [h(h(f(g(x)),g(x)),i) == i for i = 1:5]
    end
end

In [44]:
CombinedSubexpressions.cacheify!(expr)

quote 
    begin  # In[43], line 19:
        function foo(x)
            ##gx#324 = g(x)
            ##f##gx#324#325 = f(##gx#324)
            ##h##f##gx#324#325##gx#324#326 = h(##f##gx#324#325,##gx#324)
            begin  # In[43], line 20:
                [h(##h##f##gx#324#325##gx#324#326,i) == i for i = 1:5]
            end
        end
    end
end

In [45]:
@CombinedSubexpressions.cse function foo(x)
    [f(x) == i for i in 1:5]
end

begin 
    function foo(x)
        ##fx#327 = f(x)
        begin  # In[45], line 2:
            [##fx#327 == i for i = 1:5]
        end
    end
end


foo (generic function with 1 method)

In [46]:
foo(1)

f


5-element Array{Bool,1}:
 false
  true
 false
 false
 false

In [47]:
@CombinedSubexpressions.cse begin
    [h(h(f(g(x)), g(x)), i) == i for i in 1:5]
end

begin 
    ##gx#328 = g(x)
    ##f##gx#328#329 = f(##gx#328)
    ##h##f##gx#328#329##gx#328#330 = h(##f##gx#328#329,##gx#328)
    begin  # In[47], line 2:
        [h(##h##f##gx#328#329##gx#328#330,i) == i for i = 1:5]
    end
end
g
f
h
h
h
h
h
h


5-element Array{Bool,1}:
 false
 false
  true
 false
 false

In [48]:
@CombinedSubexpressions.cse begin
    x = zeros(5)
    for i in 1:5
        x[i] = f(x[i]) + g(5)
    end
end

begin 
    ##zeros5#331 = zeros(5)
    ##g5#332 = g(5)
    begin  # In[48], line 2:
        x = ##zeros5#331 # In[48], line 3:
        for i = 1:5 # In[48], line 4:
            x[i] = f(x[i]) + ##g5#332
        end
    end
end
g
f
f
f
f
f


In [49]:
@CombinedSubexpressions.cse function bar(y)
    x = zeros(5)
    for i in 1:5
        x[i] = f(x[i]) + g(5)
    end
end

function bar_original(y)
    x = zeros(5)
    for i in 1:5
        x[i] = f(x[i]) + g(5)
    end
end

begin 
    function bar(y)
        ##zeros5#333 = zeros(5)
        ##g5#334 = g(5)
        begin  # In[49], line 2:
            x = ##zeros5#333 # In[49], line 3:
            for i = 1:5 # In[49], line 4:
                x[i] = f(x[i]) + ##g5#334
            end
        end
    end
end


bar_original (generic function with 1 method)

In [50]:
@code_lowered bar(1)

LambdaInfo template for bar(y) at In[49]:2
:(begin 
        zeros5#333 = (Main.zeros)(5)
        g5#334 = (Main.g)(5)
        # meta: location In[49]
        nothing
        x = zeros5#333 # line 3:
        SSAValue(0) = (Main.colon)(1,5)
        #temp# = (Base.start)(SSAValue(0))
        9: 
        unless !((Base.done)(SSAValue(0),#temp#)) goto 19
        SSAValue(1) = (Base.next)(SSAValue(0),#temp#)
        i = (Core.getfield)(SSAValue(1),1)
        #temp# = (Core.getfield)(SSAValue(1),2) # line 4:
        SSAValue(2) = (Main.f)((Main.getindex)(x,i)) + g5#334
        (Main.setindex!)(x,SSAValue(2),i)
        17: 
        goto 9
        19: 
        # meta: pop location
        return
    end)

In [40]:
@code_lowered bar_original(1)

LambdaInfo template for bar_original(y) at In[38]:9
:(begin 
        nothing
        x = (Main.zeros)(5) # line 10:
        SSAValue(0) = (Main.colon)(1,5)
        #temp# = (Base.start)(SSAValue(0))
        6: 
        unless !((Base.done)(SSAValue(0),#temp#)) goto 16
        SSAValue(1) = (Base.next)(SSAValue(0),#temp#)
        i = (Core.getfield)(SSAValue(1),1)
        #temp# = (Core.getfield)(SSAValue(1),2) # line 11:
        SSAValue(2) = (Main.f)((Main.getindex)(x,i)) + (Main.g)(5)
        (Main.setindex!)(x,SSAValue(2),i)
        14: 
        goto 6
        16: 
        return
    end)